##

In [37]:
from matplotlib import pyplot as plt
import numpy as np
import torch

import torchvision
from torchvision import datasets, transforms

from collections import Counter
from torch.utils.data import random_split

In [38]:
seed = 10
torch.manual_seed(seed)

category_index = 8
n_val = 5000

data_path = '/cifar-10-batches-py'

Function for loading the Cifar10 dataset.

The method will have to be run twice.
After running the method for the first time we get create a normalizer from the std and mean of the images. The method is then ran for a second time with the normalizer as the preprocessor.

Loading the CIFAR-10 dataset as tensors.

In [39]:
transformed_cifar10_train_val = datasets.CIFAR10(
    data_path,
    train=True,
    download=False,
    transform = transforms.ToTensor()
)

Stacking the set of images into a single tensor. We then create a normalizer for the dataset around the mean and standard deviation of the 3 dimensions (height, width channel (color)).

In [40]:
imgs = torch.stack([img for img, _ in transformed_cifar10_train_val])

normalizer = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean = imgs.mean(dim=(0, 2, 3)), std = imgs.std(dim=(0, 2, 3)))])

Loading the dataset as tensors for training+validation and testing. This time we apply the composition of transforms.

In [41]:
normalized_cifar10_train_val = datasets.CIFAR10(
    data_path,
    train=True,
    download=False,
    transform = normalizer
)


transformed_cifar10_test = datasets.CIFAR10(
    data_path,
    train=False,
    download=False,
    transform = transforms.Compose([transforms.ToTensor(),normalizer])
)

As this is a binary classification problem where we only want to identify whether an image is a ship or not, we can set the labels that are "ship" to true. We set all other labels to false.

In [ ]:
train_labels = np.array(transformed_cifar10_train_val)
train_labels = np.array(train_labels == category_index).astype(int)

test_labels = np.array(transformed_cifar10_test)
test_labels = np.array(test_labels == category_index).astype(int)

Splitting the training and validation set randomly.

In [ ]:
n_train = len(transformed_cifar10_train_val)-n_val

transformed_cifar10_train_split, transformed_cifar10_val_split = random_split(
    transformed_cifar10_train_val,
    [n_train, n_val],

    generator=torch.Generator().manual_seed(seed)
)

print("Size of the train dataset:        ", len(transformed_cifar10_train_split))
print("Size of the validation dataset:   ", len(transformed_cifar10_val_split))
print("Size of the test dataset:         ", len(transformed_cifar10_test))

Counter([label for _, label in transformed_cifar10_train_split])

Choosing a pre-trained CNN model: we chose ResNet18, which is not trained on Cifar-10.

In [ ]:
from torchvision import models
import torch.nn as nn
import torch.optim as optim

Loading pre-trained ResNet18 model and modifying the last layer. We are doing binary classification, so we think we only need one node in the final layer.

In [ ]:
model = models.resnet18(pretrained=True)

num_features = model.fc.in_features
model.fc = nn.Linear(num_features,1)

We use Binary Cross Entropy as it should be suitable for binary classification problems (add reasoning and explanation). We use nn.BCEWithLogitsLoss() as it combines the sigmoid activation function and the BCE into a single class.

The optimizer we use is Adam, and we will begin with a learning rate of 0.001, just because it is a commonly used learning rate.

In [ ]:
loss_function = nn.BCEWithLogitsLoss()


optimizer = optim.adam(model.parameters(),  lr=0.001)